In [1]:
import os
DATA_PATH = os.path.join(os.getcwd(), 'data')
TEST_DATASET_PATH = os.path.join(DATA_PATH, 'test')
TRAIN_DATASET_PATH = os.path.join(DATA_PATH, 'train')
OUTPUT_PATH = os.path.join(DATA_PATH, 'segmented_train')
SEGMENTED_TRAIN_DATASET_PATH = os.path.join(DATA_PATH, 'segmented_train')
AUGMENTED_SEGMENTED_TRAIN_DATASET_PATH = os.path.join(DATA_PATH, 'augmented_segmented_train')

In [ ]:
import os
import cv2
import numpy as np

def adaptive_thresholding(image):
    return cv2.adaptiveThreshold(
        image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, blockSize=11, C=2
    )

def remove_noise(image):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    cleaned_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=1)
    cleaned_image = cv2.morphologyEx(cleaned_image, cv2.MORPH_CLOSE, kernel, iterations=1)
    return cleaned_image

def remove_lines(image):
    lines = cv2.HoughLinesP(image, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(image, (x1, y1), (x2, y2), 0, thickness=2)
    return image

def segment_characters(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    char_images = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if h > 10:  # Filter small noise
            char_img = image[y:y+h, x:x+w]
            char_images.append((x, char_img))
    char_images = sorted(char_images, key=lambda c: c[0])  # Sort by x-coordinate
    return [char[1] for char in char_images]

def process_and_save_images(input_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process each image in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            # Extract the true label from the filename
            base_name = filename.split('-')[0]

            # Load and preprocess the image
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Apply preprocessing steps
            image = adaptive_thresholding(image)
            image = remove_noise(image)
            image = remove_lines(image)

            # Segment characters
            char_images = segment_characters(image)

            # Validate the number of segmented parts
            if len(char_images) != len(base_name):
                print(f"Skipping {filename}: Expected {len(base_name)} parts, got {len(char_images)}.")
                continue  # Skip saving this image

            # Save each segmented character follow the naming convention <label_str>-<char>.png
            for idx, char_img in enumerate(char_images):
                char_label = base_name[idx]
                save_name = f"{base_name}-{char_label}.png"
                save_path = os.path.join(output_dir, save_name)

                # Save the segmented character
                cv2.imwrite(save_path, char_img)

    print(f"Processing complete! Segmented characters are saved in {output_dir}")


process_and_save_images(TRAIN_DATASET_PATH, SEGMENTED_TRAIN_DATASET_PATH)

Skipping 002k-0.png: Expected 4 parts, got 3.
Skipping 006aguv-0.png: Expected 7 parts, got 5.
Skipping 00fh-0.png: Expected 4 parts, got 0.
Skipping 00hgi3n7-0.png: Expected 8 parts, got 9.
Skipping 010b-0.png: Expected 4 parts, got 0.
Skipping 010ud-0.png: Expected 5 parts, got 6.
Skipping 01gokr0s-0.png: Expected 8 parts, got 10.
Skipping 01hmm-0.png: Expected 5 parts, got 6.
Skipping 026hn-0.png: Expected 5 parts, got 6.
Skipping 02cc-0.png: Expected 4 parts, got 2.
Skipping 02d69zs0-0.png: Expected 8 parts, got 4.
Skipping 02tkd-0.png: Expected 5 parts, got 7.
Skipping 03518k9-0.png: Expected 7 parts, got 10.
Skipping 03mxgd-0.png: Expected 6 parts, got 7.
Skipping 03ykb47u-0.png: Expected 8 parts, got 5.
Skipping 04eli6gc-0.png: Expected 8 parts, got 7.
Skipping 04ue-0.png: Expected 4 parts, got 0.
Skipping 053e-0.png: Expected 4 parts, got 0.
Skipping 057ea-0.png: Expected 5 parts, got 4.
Skipping 05h9r3yv-0.png: Expected 8 parts, got 10.
Skipping 05u5elo-0.png: Expected 7 parts

In [132]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_and_save_images(input_dir, output_dir, augment_count=3):
    """Augment images from `input_dir` and save them with proper naming to `output_dir`."""
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    # Define a more controlled data augmentation pipeline
    datagen = ImageDataGenerator(
        rotation_range=10,          # Small rotation for better quality
        width_shift_range=0.05,     # Slight horizontal shift
        height_shift_range=0.05,    # Slight vertical shift
        shear_range=0.1,            # Mild shearing
        zoom_range=0.1,             # Mild zoom
        fill_mode='nearest'         # Avoid introducing artifacts
    )

    # Loop through all images in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            # Load the original image
            image_path = os.path.join(input_dir, filename)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Expand dimensions for augmentation
            img = np.expand_dims(img, axis=-1)  # Add channel dimension
            img = np.expand_dims(img, axis=0)   # Add batch dimension

            # Extract the base filename without extension (e.g., 'abc-o')
            base_name = filename.split('.')[0]

            # Generate and save augmented images
            for i, batch in enumerate(datagen.flow(img, batch_size=1)):
                # Save each augmented image with proper naming
                augmented_filename = f"{base_name}-{i + 1}.png"
                augmented_path = os.path.join(output_dir, augmented_filename)

                # Save the augmented image
                cv2.imwrite(augmented_path, batch[0].squeeze().astype(np.uint8))

                # Stop after generating `augment_count` images per original image
                if i + 1 >= augment_count:
                    break

    print(f"Augmentation complete! Augmented images saved in {output_dir}")


# Example usage
input_dir = SEGMENTED_TRAIN_DATASET_PATH
output_dir = AUGMENTED_SEGMENTED_TRAIN_DATASET_PATH
augment_and_save_images(input_dir, output_dir)


Augmentation complete! Augmented images saved in /Users/jjtiong/Desktop/captcha_dataset/augmented_segmented_train


In [133]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Define character set (e.g., 0-9, a-z)
CHAR_SET = "0123456789abcdefghijklmnopqrstuvwxyz"
CHAR_TO_IDX = {char: idx for idx, char in enumerate(CHAR_SET)}

def load_dataset(data_dir):
    images = []
    labels = []

    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            # Extract the character from the filename (e.g., 'o' from 'oq1j-o.png')
            char_label = filename.split('-')[1][0]

            if char_label in CHAR_TO_IDX:
                # Load the image in grayscale
                image_path = os.path.join(data_dir, filename)
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                # Resize to 28x28 (if needed)
                img = cv2.resize(img, (28, 28))
                img = img / 255.0  # Normalize pixel values to [0, 1]

                # Store the image and label
                images.append(img)
                labels.append(CHAR_TO_IDX[char_label])

    # Convert to numpy arrays
    images = np.array(images).reshape(-1, 28, 28, 1)  # Add channel dimension
    labels = np.array(labels)

    return images, labels

# Load the dataset
images, labels = load_dataset(output_dir)
print(f"Loaded {len(images)} images with {len(set(labels))} unique labels.")

Loaded 51273 images with 36 unique labels.


In [134]:
# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}, Validation set size: {X_val.shape[0]}")


Training set size: 41018, Validation set size: 10255


In [135]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model(num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Build the model for 36 classes (0-9, a-z)
num_classes = len(CHAR_SET)
model = build_model(num_classes)
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 36)             │         4,644 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 228,388 (892.14 KB)

 Trainable params: 228,388 (892.14 KB)

 Non-trainable params: 0 (0.00 B)

In [137]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64
)

Epoch 1/30
641/641 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.1906 - loss: 3.0263 - val_accuracy: 0.4712 - val_loss: 1.9854
Epoch 2/30
641/641 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.4253 - loss: 2.1478 - val_accuracy: 0.5188 - val_loss: 1.7630
Epoch 3/30
641/641 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.4659 - loss: 1.9785 - val_accuracy: 0.5459 - val_loss: 1.6650
Epoch 4/30
641/641 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.4960 - loss: 1.8650 - val_accuracy: 0.5639 - val_loss: 1.6066
Epoch 5/30
641/641 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.5158 - loss: 1.7770 - val_accuracy: 0.5720 - val_loss: 1.5537
Epoch 6/30
641/641 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.5292 - loss: 1.7416 - val_accuracy: 0.5780 - val_loss: 1.5223
Epoch 7/30
641/641 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.5357 - loss: 1.6931 - val_accuracy: 0.5878 - val_loss: 1.4963
Epoch 8/30
641/641 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.5430 - loss: 1.6647 - val_accu

In [138]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%")


321/321 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6330 - loss: 1.2839
Validation Loss: 1.2811, Validation Accuracy: 63.69%


In [139]:
# Save the trained model
def save_model(model, dir_path, model_name="character_recognition_model.h5"):
    # Ensure the directory exists
    os.makedirs(dir_path, exist_ok=True)
    
    # Create the full path to the model file
    model_path = os.path.join(dir_path, model_name)
    
    # Save the model
    model.save(model_path)
    print(f"Model saved successfully at: {model_path}")

# Example usage
save_dir = "/Users/jjtiong/Desktop/captcha_dataset/model"
save_model(model, save_dir)

Model saved successfully at: /Users/jjtiong/Desktop/captcha_dataset/model/character_recognition_model.h5


In [140]:
def predict_character(image_path, model):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28)) / 255.0  # Normalize
    img = np.expand_dims(img, axis=[0, -1])  # Add batch and channel dimensions

    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    predicted_char = CHAR_SET[predicted_label]

    return predicted_char

# Example usage
model = tf.keras.models.load_model('/Users/jjtiong/Desktop/captcha_dataset/model/character_recognition_model.h5')
predicted_char = predict_character('/Users/jjtiong/Desktop/captcha_dataset/segmented_train/0tqk-t.png', model)
print(f"Predicted Character: {predicted_char}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted Character: t


In [141]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model_path = '/Users/jjtiong/Desktop/captcha_dataset/model/character_recognition_model.h5'
model = load_model(model_path)

# Define the character set used during training
CHAR_SET = "0123456789abcdefghijklmnopqrstuvwxyz"
CHAR_MAP = {idx: char for idx, char in enumerate(CHAR_SET)}

def preprocess_image(image_path):
    """Preprocess the image using the same steps as in training."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = adaptive_thresholding(image)
    image = remove_noise(image)
    image = remove_lines(image)
    return image

def adaptive_thresholding(image):
    """Apply adaptive thresholding."""
    return cv2.adaptiveThreshold(
        image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, blockSize=11, C=2
    )

def remove_noise(image):
    """Remove small noise using morphological operations."""
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    cleaned_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=1)
    cleaned_image = cv2.morphologyEx(cleaned_image, cv2.MORPH_CLOSE, kernel, iterations=1)
    return cleaned_image

def remove_lines(image):
    """Remove lines using Hough Line Transform."""
    lines = cv2.HoughLinesP(image, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(image, (x1, y1), (x2, y2), 0, thickness=2)
    return image

def segment_characters(image):
    """Segment the CAPTCHA image into individual characters."""
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    char_images = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if h > 10:  # Ignore small contours (noise)
            char_img = image[y:y+h, x:x+w]
            char_images.append((x, char_img))

    # Sort by x-coordinate to ensure correct order
    char_images = sorted(char_images, key=lambda c: c[0])
    return [char[1] for char in char_images]

def predict_character(image, model):
    """Predict a single character using the trained model."""
    img = cv2.resize(image, (28, 28)) / 255.0  # Resize and normalize
    img = np.expand_dims(img, axis=[0, -1])  # Add batch and channel dimensions

    prediction = model.predict(img)
    predicted_label = np.argmax(prediction)
    predicted_char = CHAR_MAP[predicted_label]
    return predicted_char

def predict_captcha_string(image_path, model):
    """Predict the entire CAPTCHA string from the input image."""
    # Preprocess the image
    processed_image = preprocess_image(image_path)

    # Segment the characters
    char_images = segment_characters(processed_image)

    # Predict each character and reconstruct the string
    predicted_string = ""
    for char_img in char_images:
        predicted_char = predict_character(char_img, model)
        predicted_string += predicted_char

    return predicted_string



# Example usage
captcha_image_path = "/Users/jjtiong/Desktop/captcha_dataset/test/2d76-0.png"
predicted_string = predict_captcha_string(captcha_image_path, model)
print(f"Predicted CAPTCHA string: {predicted_string}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Predicted CAPTCHA string: ljd76


In [129]:
import os

def evaluate_on_unseen_data(unseen_dir, model, num_images=20):
    """Evaluate the model on the first `num_images` from the unseen directory."""
    correct_predictions = 0
    total_images = 0

    # Get the first `num_images` files from the directory
    all_files = [f for f in os.listdir(unseen_dir) if f.endswith('.png')]
    all_files = sorted(all_files)[:num_images]  # Limit to first `num_images`

    for filename in all_files:
        # Extract the true label from the filename (e.g., '2d76' from '2d76-0.png')
        true_label = filename.split('-')[0]

        # Get the full path to the image
        image_path = os.path.join(unseen_dir, filename)

        # Predict the CAPTCHA string
        predicted_string = predict_captcha_string(image_path, model)
        print(f"True: {true_label}, Predicted: {predicted_string}")

        # Compare the predicted string with the true label
        if predicted_string == true_label:
            correct_predictions += 1

        total_images += 1

    # Calculate accuracy
    accuracy = correct_predictions / total_images * 100
    print(f"\nAccuracy on the first {num_images} images: {accuracy:.2f}%")

# Example usage
unseen_dir = "/Users/jjtiong/Desktop/captcha_dataset/test"
evaluate_on_unseen_data(unseen_dir, model, 1000)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
True: 002e23, Predicted: 00ze23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
True: 03yl9s, Predicted: o2iyi9s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
True: 03yuav5, Predicted: 09kvdv5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━